### Notes for this jupyternotebook: 
This Notebook can run without the RFSoC to generates the `.mem` files, here's some steps that I took 
1. Create a venv ``python3 -m venv <name of your venv``
2. Install neccessary packages such as ``matplotlib`` and ``numpy`` using ``pip3 install``
3. Install QICK modules, runing `pip3 install git+https://github.com/openquantumhardware/qick.git`
4. Be sure you have the `qick_config_216.json` for `soccfg`

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import json

import qick 
# help(qick)
from qick import *
from qick.asm_v2 import AveragerProgramV2
from qick.asm_v2 import QickSpan, QickSweep1D
from qick import QickConfig

In [2]:
soccfg = QickConfig("qick_config_216.json")

QICK library version mismatch: 0.2.366 remote (the board), 0.2.369 local (the PC)
                        This may cause errors, usually KeyError in QickConfig initialization.
                        If this happens, you must bring your versions in sync.


In [6]:
GEN_CH = 0
RO_CH = 0
TRIG_TIME = 0.40
FREQ = 100
# FREQ = 0
class MultiPulseProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        
        self.declare_gen(ch=gen_ch, nqz=1)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])

        ramp_len = self.cycles2us(20, gen_ch=gen_ch)
        self.add_gauss(ch=gen_ch, name="ramp", sigma=ramp_len/10, length=ramp_len, even_length=True)
        
        self.add_pulse(ch=gen_ch, name="myflattop", ro_ch=ro_ch, 
                       style    = "flat_top", 
                       envelope = "ramp", 
                       freq     = cfg['freq'], 
                       length   = 0.1,
                       phase    = 0,
                       gain     = 1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="myflattop2", ro_ch=ro_ch, 
                       style    = "flat_top", 
                       envelope = "ramp", 
                       freq     = cfg['freq'], 
                       length   = 0.1,
                       phase    = 90,
                       gain     = 0.5, 
                      )

        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['freq'], gen_ch=gen_ch)
        # send the config to the dynamic RO
        self.send_readoutconfig(ch=ro_ch, name="myro", t=0)
        
    def _body(self, cfg):
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'], ddr4=True)

        for i in range(0,15):
            self.pulse(ch=cfg['gen_ch'], name="myflattop", t=1.0)
            self.pulse(ch=cfg['gen_ch'], name="myflattop2", t=1.0)

In [8]:
config = {'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'freq': FREQ,
          'trig_time': 1.0,
          'ro_len': 5.0,
         }

In [9]:
prog = MultiPulseProgram(soccfg, reps=1, final_delay=0.5, cfg=config)

In [17]:
prog.print_pmem2hex()

// PMEM content
000000000000000000
8c600000000000000c
dce000c20000000000
4c080000000000d700
8c6000000000000020
dda00030000000d700
dda00034800000d700
dda00035000000d700
dda00010000000dc00
dda00014800000dc00
dda00015000000dc00
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d700
dce000a0000000d700
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d700
dce000a0000000d700
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d700
dce000a0000000d700
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d700
dce000a0000000d700
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d700
dce000a0000000d700
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d700
dce000a0000000d700
dce00000000000d700
dce00020000000d700
dce00040000000d700
dce00060000000d700
dce00080000000d

In [20]:
prog.print_wmem2hex()
# prog.print_sg_mem(sg_idx=0,gen_file=True)

// WMEM content
// CONF_     LEN_    GAIN_   ENV_   PHASE_    FREQ
___0008_0000000a_00007ffe_000000_00000000_02abc2bc
___0009_0000003c_00003fff_000000_00000000_02abc2bc
___0008_0000000a_00007ffe_00000a_00000000_02abc2bc
___0008_0000000a_00003ffe_000000_40000000_02abc2bc
___0009_0000003c_00001fff_000000_40000000_02abc2bc
___0008_0000000a_00003ffe_00000a_40000000_02abc2bc


OverflowError: Python integer -174762666 out of bounds for uint32

In [21]:
prog.print_sg_mem(sg_idx=0,gen_file=True)

AttributeError: 'MultiPulseProgram' object has no attribute 'print_sg_mem'